In [2]:
import timeit
from math import sqrt, sin, cos, pi, atan2
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
import matplotlib as mpl
plt.style.use('classic')
plt.rc('text', usetex=True)
plt.rcParams["figure.figsize"] = (10,8)
from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider
import ipywidgets as widgets

# Motion of a Pendulum under damping (air drag) and a driving force:
### Mark Durgin, and DJ Pelletier
#### Report Due: May 4th, 2020 5pm

Consider a pendulum consisting of a dense sphere of negligible radius, at the end of a very
light rigid stick.<br>
Assume that the end of the stick is attached to a frictionless pivot that allows the pendulum to rotate<br>
through $2\pi$ radians.<br>
The pendulum is subject to the force of air resistance, acting opposite the angular velocity.<br>
It is further subject to a sinusoidal driving force (a force applied to the pendulum that repeats itself with a certain frequency).<br>
This project is designed to use python to investigate such a pendulum, and <br>
to see the following:
1. the difference between the Euler-Cromer, and 2nd Order Runge-Kutta methods when applied to this system
2. experimentally determine the damping constant at which critical damping occurs, that is, the air drag force slows down and brings motion to a stop
3. experimentally determine the resonance Amplitude while the system is opertaing under the driving force.

## Physical Description of the System
------
<img src="Pendulum-444444.svg" width="280" align="center"/>
To simulate the motion of the pendulum, we need only look to Newton's second law. As well, given that the path the pendulum mass can follow is a circle, it makes the most sense to use 2D polar coordinates L and $\theta$. Therefore in particular we look to Newton's second law for torques:
$$
\sum \vec{\tau} = I\ddot{\vec{\theta}}
$$
Since the tension in the rod is radial, it can't apply a torque. Then $\sum \vec{\tau}$ is the torque due to the forces acting on the mass in the perpendicular direction:  Those are gravity, a linear air drag force and a driving force.  The sum of these forces as shown above, is 

$$
\tau = I\ddot{\theta} = -mgLsin(\theta) - q\dot{\theta} + F_d
$$

Where $q$ is the damping constant and $F_d$ represents a driving force applied to the pendulum.

In our General Physics course-work, we consider circumstances where the pendulum oscillates within the confines of a small or "finite" angle (usually $25^{\circ}$ or less), in which case we could allow $sin(\theta) = \theta$. For our purposes here, it makes a good starting point.  We start by isolating the $\ddot{\theta}$ component, dividing each term by the Moment of Inertia.  In this case, we can approximate the pendulum mass as a point-mass, hence the Moment of Inertia $I = mL^2$ , the ball's mass times the radius (the stick lenth) squared: 

$$
\begin{align*}
\ddot{\theta} &= - \frac{mgL}{mL^2}\theta -\frac{q}{mL^2}\dot{\theta} + \frac{F_d}{mL^2}\\
\end{align*}
$$

and we modify this to a more useful form:

$$
\begin{align*}
\ddot{\theta} &= -\frac{1}{mL^2}(gmL\theta + q\dot{\theta} - F_d)\\
\end{align*}
$$


We will now deploy the Euler-Cromer method to work with this problem computationally.  We note that it can be rewritten in terms of $\ddot{\theta} = \frac{d\omega}{dt}$ and $\dot{\theta} = \omega$ , as follows:
$$
\frac{d\omega}{dt} = -\frac{1}{mL^2}(gmL\theta + q\omega - F_d)
$$
We view the above equation as a small change over a finite time interval, rather than as a true derivative; then we can say:
$$
\frac{\Delta \omega}{\Delta t} = -\frac{1}{mL^2}(gmL\theta + q\omega - F_d)\\
$$
Letting our $\theta$'s and $\omega$'s be represented by time steps $i+1$ and $i$, we see:
$$
\begin{align*}
\omega_{i+1} - \omega_i &= -\frac{1}{mL^2}(gmL\theta_i + q\omega_i - F_d)\Delta t \\
\omega_{i+1} &= \omega_i -\frac{1}{mL^2}(gmL\theta_i + q\omega_i - F_d)\Delta t \\
\end{align*}
$$
<br>
By the same token, we obtain an expression for the next step in $\theta$ (again, using Euler-Cromer):
$$
\begin{align*}
\frac{d\theta}{dt} &= \omega \\
\frac{\Delta \theta}{\Delta t} &= \omega_{i+1} \\
\theta_{i+1} - \theta_i &= (\omega_{i+1})\Delta t \\
\theta_{i+1} &= \theta_i + (\omega_{i+1})\Delta t
\end{align*}
$$


### The below function usies the finite angle approximation:

In [3]:
def run_sim_pendulum(Δt, θ_0, ω_0, q, Ω_d, F_d, m, L, tf, method, g=9.81):
    """
    Given a choice of several computational methods, run a simulation of a pendulum under 
    3 forces: 
    - The force due to gravity close to the surface of the Earth,
    - a damping force (such as air resistance) and
    - a Driving Force, such as an electrical impulse.
    Regardless of the choice of method, the finite angle approximation is used in deriving the equations of motion.
    
    ARGUMENTS:
    ----------
        Δt: The time increment to be used, in seconds.
        θ_0: The initial angle value in degrees.
        ω_0: The initial angular velocity in rad/s.
        q:   The damping constant, in N*s/m (Newston-seconds over meters)
        Ω_d: The driving force frequency, in 1/sec
        F_d:  The applied driving force, in N
        m: The mass of the pendulum ball, in kg.
        L: The length of the pendulum, in meters.
        n:  The amount of time to run the simulation, in seconds
        method: One of either 'ec', or 'rk2', denoting the Euler-Cromer or 2nd order
                Runge-Kutta methods, respectively.
    
    RETURNS:
    --------
        θ: List containing the angle values the pendulum went through, in radians.
        ω: List containing the angular velocity values the pendulum went through, in rad/s.
        time: List containing time values, in seconds.
    """
    Ω_o = np.sqrt(g/L) # Natural frequency
    # T = 2*np.pi*np.sqrt(L/g)   # Natural Period of the pendulum with no damping or externL forces.
    θ = [np.radians(θ_0)]
    ω = [ω_0]
    time = [0]
    
    
    
    if method == 'ec':
        while (time[-1] <= tf):
            
            ω.append(ω[-1] - 1/(m*(L**2))*(g*m*L*θ[-1] + q*ω[i] - F_d*np.sin(Ω_d*time[-1]))*Δt) 
            θ.append(θ[-1] + ω[-1]*Δt)
            
            time.append(time[-1] + Δt)
            
            
            
    elif method == 'rk2':
        while (time[-1] <= tf):
            
            k1 = ω[-1]*Δt
            m1 = (-g/L*θ[-1])*Δt
            k2 = (ω[-1] + m1)*Δt
            θ.append(θ[-1] + 0.5*(k1 + k2))
            m2 = (-g/L*(θ[-1]))*Δt
            ω.append(ω[-1] + 0.5*(m1 + m2))
            
            time.append(time[-1] + Δt)
            
            energy.append(m*g*L*(1-np.cos(θ[-1])) + 0.5*m*(L*ω[-1])**2)
            
            
    
    return θ, ω, time